# IE 582 Homework 03 - Dorukhan Kılınç 2017402093

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score
import timeit

In [2]:
#Read train data
file_names = ["uWaveGestureLibrary_X_TRAIN",
              "uWaveGestureLibrary_Y_TRAIN",
              "uWaveGestureLibrary_Z_TRAIN"]

data_train = [[], [], []]

for i in range(3):
    for line in open(file_names[i]):
        data_train[i].append(line.split())
    data_train[i] = np.array(data_train[i])
    
data_acc_train = np.ndarray(shape = (896, 316, 3), dtype = "float")

for i in range(3):
    data_acc_train[:,:, i] = data_train[i]

data_v_train = np.zeros(shape = (896, 316, 3))

data_v_train[:,0,:] = data_acc_train[:,0,:]

for instance in range(896):
    for time in range(2, 316):                   
        data_v_train[instance, time, :] = data_v_train[instance, time - 1,:] + data_acc_train[instance, time - 1,:]
        
data_dist_train = np.zeros(shape = (896, 316, 3))
data_dist_train[:,0,:] = data_acc_train[:,0,:]

for instance in range(896):
    for time in range(2, 316):                   
        data_dist_train[instance, time, :] = data_dist_train[instance, time - 1,:] + data_v_train[instance, time - 1,:] + data_acc_train[instance, time - 1,:]/2


In [3]:
#Read test data

file_names2 = ["uWaveGestureLibrary_X_TEST",
              "uWaveGestureLibrary_Y_TEST",
              "uWaveGestureLibrary_Z_TEST"]

data_test = [[], [], []]

for i in range(3):
    for line in open(file_names2[i]):
        data_test[i].append(line.split())
    data_test[i] = np.array(data_test[i])
    
data_acc_test = np.ndarray(shape = (3582, 316, 3), dtype = "float")

for i in range(3):
    data_acc_test[:,:, i] = data_test[i]

data_v_test = np.zeros(shape = (3582, 316, 3))

data_v_test[:,0,:] = data_acc_test[:,0,:]

for instance in range(3582):
    for time in range(2, 316):                   
        data_v_test[instance, time, :] = data_v_test[instance, time - 1,:] + data_acc_test[instance, time - 1,:]
        
data_dist_test = np.zeros(shape = (3582, 316, 3))
data_dist_test[:,0,:] = data_acc_test[:,0,:]

for instance in range(3582):
    for time in range(2, 316):                   
        data_dist_test[instance, time, :] = data_dist_test[instance, time - 1,:] + data_v_test[instance, time - 1,:] + data_acc_test[instance, time - 1,:]/2


## Task 1

### Part a

In [33]:
#Prepare the concatanated distances
data_dist_test_x = data_dist_test[:,1:,0]
data_dist_test_y = data_dist_test[:,1:,1]
data_dist_test_z = data_dist_test[:,1:,2]

data_dist_test_concatenated = []

for instance in range(3582):
    data_dist_test_concatenated.append(np.concatenate((data_dist_test_x[instance,:],
                                           data_dist_test_y[instance,:],
                                           data_dist_test_z[instance,:])))


In [52]:
#Apply PCA to the concatanated series
#pca_concatanated = PCA(n_components=2)
#X = np.array(pca_concatanated.fit_transform(data_dist_test_concatenated))
X = np.array(data_dist_test_concatenated)

y = data_dist_test[:,0,0]

#k_values = np.arange(1, int(np.sqrt(3582)))
k_values = np.arange(2, 11)


cv_results_p1 = []
cv_results_p2 = []


for k in k_values:
    
    classifier1 = KNeighborsClassifier(n_neighbors=k, p = 1)
    cv1 = cross_validate(classifier1, X, y, cv=10)
    
    classifier2 = KNeighborsClassifier(n_neighbors=k, p = 2)
    cv2 = cross_validate(classifier2, X, y, cv=10)
    
    cv_results_p1.append(cv1)
    cv_results_p2.append(cv2)




In [53]:
cv_means_p1 = np.zeros(len(k_values))
cv_stds_p1 = np.zeros(len(k_values))

cv_means_p2 = np.zeros(len(k_values))
cv_stds_p2 = np.zeros(len(k_values))


for i in range(len(k_values)):
    
    cv_means_p1[i] = np.mean(cv_results_p1[i]["test_score"])
    cv_stds_p1[i] = np.std(cv_results_p1[i]["test_score"])
    
    cv_means_p2[i] = np.mean(cv_results_p2[i]["test_score"])
    cv_stds_p2[i] = np.std(cv_results_p2[i]["test_score"])
    
k_p1 = k_values[np.argmax(cv_means_p1)]
k_p2 = k_values[np.argmax(cv_means_p2)]

print(k_p1)
print(k_p2)

3
3


Here in this part, I used 10-fold cross-validation to find the best k value to use in the nearest neighbor classifier. Before that, I concetenated three axes such that I provided vectors of lenght 3T to the classifiers. I chose 2 different distance measures from the Minkowski family, with p = 1(Manhattan distance) and with p = 2(Euclidean distance). 

### Part b

In [54]:
#Timing 1st classifier, i.e. 3-nearest-neighbor classifier with manhattan distance measure
start_p1 = timeit.default_timer()
classifier1 = KNeighborsClassifier(n_neighbors=k_p1, p = 1)
classifier1.fit(X, y)
stop_p1 = timeit.default_timer()
time_p1 = stop_p1 - start_p1

#Timing 2nd classifier, i.e. 3-nearest-neighbor classifier with euler distance measure
start_p2 = timeit.default_timer()
classifier2 = KNeighborsClassifier(n_neighbors=k_p2, p = 2)
classifier2.fit(X, y)
stop_p2 = timeit.default_timer()
time_p2 = stop_p2 - start_p2


In [55]:
print("Accuracy for model 1 is "+str(classifier1.score(X,y)) +"\n")
print("Run time for model 1 is "+str(time_p1) +"\n")
print("Confusion Matrix:")
print(confusion_matrix(y, classifier1.predict(X)))

Accuracy for model 1 is 0.966499162479062

Run time for model 1 is 0.003537389999110019

Confusion Matrix:
[[432   0   0   2   0   3   0   0]
 [  2 443   0   0   0   0   5   2]
 [  0   2 441   0   3   6   2   0]
 [  1   0   1 438   8   2   0   0]
 [  3   0   8   1 420   0   0   1]
 [ 11   0   4   8   7 418   1   0]
 [  3  23   2   0   0   0 416   3]
 [  0   1   0   3   0   0   2 454]]


In [56]:
print("Accuracy for model 2 is "+str(classifier2.score(X,y)) +"\n")
print("Run time for model 2 is "+str(time_p2) +"\n")
print("Confusion Matrix:")
print(confusion_matrix(y, classifier2.predict(X)))

Accuracy for model 2 is 0.9637074260189838

Run time for model 2 is 0.0028684499993687496

Confusion Matrix:
[[434   0   0   2   0   1   0   0]
 [  3 442   0   0   0   0   5   2]
 [  0   2 442   0   3   5   2   0]
 [  1   0   1 436  10   2   0   0]
 [  3   0   7   0 422   0   0   1]
 [ 10   0   5   8  10 415   1   0]
 [  2  30   1   0   0   1 410   3]
 [  0   4   0   3   0   0   2 451]]


### Part c

Weighting the distance based on the axes is a reasonable idea because the data consists of the coordinates of the movements in 3-d and the classes are represented in 2-d, which means that the movement in one of the axes will be useless. For example, in the cyclic motions, one axis will be the noise in the motion, probably the shaking of the accelometer while collecting the data. We can perform cross validation on the data to see if the useless axis is consistent and assign to it a very small coefficient or even zero. Also notice that this idea seems to be similar to PCA in the sense that we would reduce the dimensionality of the data if we would choose not to include an axis in the distance calculation. However, it is important to note that this suggestion assumes the noise axis is consistent in each instance so that we can remove it and not lose much information.

## Task 2

In [10]:
#Prepare the concatanated distances
data_dist_train_x = data_dist_train[:,1:,0]
data_dist_train_y = data_dist_train[:,1:,1]
data_dist_train_z = data_dist_train[:,1:,2]

data_dist_train_concatenated = []

for instance in range(896):
    data_dist_train_concatenated.append(np.concatenate((data_dist_train_x[instance,:],
                                           data_dist_train_y[instance,:],
                                           data_dist_train_z[instance,:])))


### Part a

In [11]:
X_train = MinMaxScaler().fit_transform(data_dist_train_concatenated)

y_train = data_dist_train[:,0,0] 

y_train_binary = (y_train == 3)*1

model = LogisticRegression(random_state=0, max_iter = 100000).fit(X_train, y_train_binary)


In [51]:
X_test = MinMaxScaler().fit_transform(data_dist_test_concatenated)

y_test = data_dist_test[:,0,0] 

y_test_binary = (y_test == 3)*1

probs = model.predict_proba(X_test)[:,1]

class3_train_ratio = sum(y_train_binary)/len(y_train_binary)

y_predicted = (probs >= class3_train_ratio)

print(confusion_matrix(y_test_binary, y_predicted))
print(accuracy_score(y_test_binary, y_predicted))

[[2922  206]
 [  41  413]]
0.9310441094360692


Here above is a simple logistic regression model classifying whether one instance belongs to the class 3 or not. For decision threshold, I used the ratio of the class 3 instances in the training data. And I plotted the confusion matrix and printed the accuracy for this model. Below, I will try to improve this model by introducing penalties and different representations.

### Part b

In [57]:
C_values = np.arange(1, 30,1)

cv_model_task2 = LogisticRegressionCV(cv=10,max_iter = 10000, penalty = "l1", scoring = "neg_log_loss",
                    solver = "liblinear", Cs = C_values,random_state=0)

cv_model_task2.fit(X_train, y_train_binary)

cv_probs = cv_model_task2.predict_proba(X_test)[:,1]


y_cv_predicted = (cv_probs >= class3_train_ratio)*1

print(confusion_matrix(y_test_binary, y_cv_predicted))
print(accuracy_score(y_test_binary, y_cv_predicted))

[[2947  181]
 [  48  406]]
0.9360692350642099


In [23]:
cv_model_task2.C_

array([12])

In [24]:
metrics.log_loss(y_test_binary, y_cv_predicted)

2.2081307988747665

Here above is a penalized version of logistic regression, with l1 penalty. The library scikitlearn has a parameter C which corresponds to the inverse of the $\lambda$ parameter. After 10-fold cross-validation, the best C is found equal to 12 and accuracy has improved. Now, lets look at the regression coefficients and try to interpret them:

In [81]:
cv_model_task2.coef_

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+0

Here above is the array of coefficients. Notice that for each axis, first half of the coefficients is 0, meaning the first half is not that important when classifying. This makes sense since distance is a cumulative sum of velocity and as the time progresses distance becomes more and more specific to each movement.

### Part c

In [26]:
print(X_train.shape)
print(X_test.shape)



(896, 945)
(3582, 945)


In [27]:
dist_train = euclidean_distances(X_train, X_train)
dist_test = euclidean_distances(X_test, X_train)
        

In [82]:
cv_model_task2c = LogisticRegressionCV(cv=10,max_iter = 10000, penalty = "l1", scoring = "neg_log_loss",
                    solver = "liblinear", Cs = C_values,random_state=0)

cv_model_task2c.fit(dist_train, y_train_binary)

cv_probs_c = cv_model_task2c.predict_proba(dist_test)[:,1]


y_cv_predicted_c = (cv_probs_c >= class3_train_ratio)*1

print(confusion_matrix(y_test_binary, y_cv_predicted_c))
print(accuracy_score(y_test_binary, y_cv_predicted_c))

[[3075   53]
 [  16  438]]
0.9807370184254607


In [29]:
cv_model_task2c.C_

array([29])

Here above is a penalized version of logistic regression applied to the distance matrices of the observations, with l1 penalty. After 10-fold cross-validation, the best C is found equal to 29 and accuracy has improved even more. This could be because now we are classifying based on how similar a test instance to the train instances. Now, lets look at the regression coefficients and try to interpret them:

In [85]:
cv_model_task2c.coef_

array([[ 8.44921553e-03,  8.37183463e-03,  9.21082796e-03,
        -7.78713335e-01, -2.19259593e-05, -9.58863553e-03,
        -3.20219804e-02, -1.34306960e-02, -2.67427368e-02,
         5.99056502e-04,  5.76109866e-02,  2.01561701e-02,
        -1.35883539e-01, -3.28186177e-02, -2.30273912e-02,
         3.00786032e-03,  8.09152126e-02, -1.01932137e-01,
         5.49412934e-03,  8.09525218e-05,  0.00000000e+00,
        -2.54969333e-02, -1.31485877e-02,  3.65067847e-02,
        -1.15733222e-02,  1.60954996e-01, -2.10241304e-01,
        -1.27733405e-02,  5.84961828e-02,  3.96635342e-01,
         2.47257181e-02,  0.00000000e+00, -4.67333439e-03,
         2.94763350e-03,  2.60676520e-01,  0.00000000e+00,
        -8.22465093e-02, -2.63901129e-02, -4.53386730e-02,
         4.62833209e-02,  1.43212543e-02,  1.14918456e-01,
         8.56095432e-02, -1.00935085e-02,  1.45735001e-04,
        -4.06767781e-02,  1.87231287e-02,  0.00000000e+00,
         6.67765258e-02,  3.36330905e-02, -2.61075698e-0

By looking at this array, we can say that almost every observation in the training set are important in classification based on the distance information with a few with 0 coefficients. About these 0's, we can say that they are not as important as the other observations in a sense that maybe they are outliers in terms of class-3 classification. Notice that the accuracy score has improved more. 

### Part d

In this task, each improvement made the model better in terms of accuracy score as I have printed each one of them. Firstly, I started with a simple model. Then, I added a penalization to the model, which resulted in a better feature selection and therefore improved the test score. Lastly, I introduced another representation while keeping the penalization and it turned out that the new representation is better since the test score improved even more.